In [3]:
from tensorflow.keras.applications import InceptionV3
from tensorflow import keras
from tensorflow.keras.layers import *
from prepare_data import *
from datetime import datetime
from tensorflow.keras.applications.inception_v3 import preprocess_input
import albumentations as A
import tensorflow as tf

In [4]:
dataset = Dataset.carabid
current_time = datetime.now().strftime("%d%m%Y-%H%M%S")
raw_train, raw_val = prep_data_single(dataset, 8)

Found 63364 files belonging to 291 classes.
Using 53860 files for training.
Found 63364 files belonging to 291 classes.
Using 9504 files for validation.


In [5]:
train_gen, val_gen = prep_SGB_dataset(dataset, raw_train, raw_val, 8)
del raw_train
del raw_val

In [6]:
standard_model_path = "models/carabid/CARABID-EXTRACTOR/extractor/savefile.hdf5"
standard_extractor = keras.models.load_model(standard_model_path).layers[0].layers[-1]
standard_extractor.trainable = False

gray_model_path = "models/gray/Dataset.carabid_12042022-162233/extractor/savefile.hdf5"
gray_extractor = keras.models.load_model(gray_model_path).layers[0].layers[-1]
gray_extractor.trainable = False


blur_model_path = "models/blur/Dataset.carabid_13042022-101814/extractor/savefile.hdf5"
blur_extractor = keras.models.load_model(blur_model_path).layers[0].layers[-1]
blur_extractor.trainable = False

In [9]:
concat_layer = concatenate([standard_extractor.output, gray_extractor.output, blur_extractor.output])
reshape_layer = Reshape((3, 1000), input_shape=(3000,))(concat_layer)
gru_layer = GRU(1000)(reshape_layer)
rnn_layer = Bidirectional(gru_layer)
dropout_layer = Dropout(0.5)(rnn_layer)
dense_layer = Dense(1000, activation='relu')
softmax = Dense(train_gen.num_classes(), activation='softmax')(dense_layer)

ValueError: Please initialize `Bidirectional` layer with a `tf.keras.layers.Layer` instance. Received: KerasTensor(type_spec=TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None), name='gru_2/PartitionedCall:0', description="created by layer 'gru_2'")

In [ ]:
classifier_model = keras.Model(inputs=[standard_extractor.input, gray_extractor.input, blur_extractor.input], outputs=softmax)
classifier_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(classifier_model.summary())

In [ ]:
logdir = "logs/unfiltered/{0}_{1}/classifier".format(str(dataset), current_time)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model_path = "models/unfiltered/{0}_{1}/classifier/savefile.hdf5".format(str(dataset), current_time)
model_save_callback = keras.callbacks.ModelCheckpoint(filepath=model_path, save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

In [ ]:
class ShuffleCallback(keras.callbacks.Callback):
    def __init__(self, generator):
        self._generator = generator
        
    def on_epoch_end(self, epoch, logs=None):
        self._generator.shuffle()
    
train_shuffle_callback = ShuffleCallback(train_gen)
val_shuffle_callback = ShuffleCallback(val_gen)

In [ ]:
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
classifier_model.fit(train_gen, validation_data=val_gen, callbacks=[tensorboard_callback, model_save_callback, train_shuffle_callback, val_shuffle_callback], epochs=20)